# Vasculature simulation use-case

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
status = be.reset_scene()

### Load vasculature

In [ ]:
assembly_name = 'Vasculature'
population_name = 'Vasculature'
be.remove_assembly(assembly_name)
vasculature_assembly = be.add_assembly(assembly_name)
vasculature_model = be.add_vasculature(
    assembly_name=assembly_name,
    population_name=population_name,
    use_sdf=False,
    quality=be.GEOMETRY_QUALITY_MEDIUM
)

In [ ]:
be.reset_camera()
core.set_renderer()

In [ ]:
be.set_vasculature_color_scheme(
    assembly_name=assembly_name,
    color_scheme=be.VASCULATURE_COLOR_SCHEME_SECTION,
    palette_name='rainbow'
)
core.set_renderer()

In [ ]:
vasculature_info = be.get_vasculature_info(assembly_name=assembly_name)
print(vasculature_info)

In [ ]:
status = core.set_renderer(
    current='bio_explorer',
    subsampling=4, max_accum_frames=64)
params = core.BioExplorerRendererParams()
params.gi_distance = 1000.0
params.shadows = 1.0
params.soft_shadows = 1.0
status = core.set_renderer_params(params)

In [ ]:
vasculature_info = be.get_vasculature_info(assembly_name=assembly_name)
print(vasculature_info)
vasculature_model_id = vasculature_info['modelId']

In [ ]:
import seaborn as sns
print(vasculature_model_id)
material_ids = be.get_material_ids(vasculature_model_id)['ids']
palette = sns.color_palette('Set1', len(material_ids))
palette[0]= [0.75, 0.75, 0.75]
be.set_materials_from_palette(
    model_ids=[vasculature_model_id],
    material_ids=material_ids,
    palette=palette, specular_exponent=5.0,
    shading_mode=be.SHADING_MODE_NONE,
    user_parameter=0.001, glossiness=1.0,
    cast_user_data=True
)
core.set_renderer()

## Simulation reports

In [ ]:
from bioexplorer import TransferFunction
tf = TransferFunction(
    bioexplorer=BioExplorer(url), name='OrRd_r',
    model_id=vasculature_model_id, size=16, alpha=1.0
)

In [ ]:
''' Attach report to vasculature '''
report_id = 0
be.set_vasculature_report(
    assembly_name=assembly_name,
    report_simulation_id=report_id,
    population_name=population_name)
if report_id % 3 == 0:
    tf.set_range([-1.16881, 5.0])
elif report_id % 3 == 1:
    tf.set_range([0.000133, 0.0008])
else:
    tf.set_range([0, 50.0])
core.set_renderer()

## Radius report

In [ ]:
import time

core.set_renderer(
    current='bio_explorer',
    subsampling=1, max_accum_frames=1)

for frame in range(100):
    core.set_animation_parameters(current=frame)
    be.set_vasculature_radius_report(
        assembly_name=assembly_name,
        population_name=population_name,
        report_simulation_id=report_id,
        frame=frame,
        amplitude=1.0
    )
    core.set_renderer()